## Early Modern London -- 
    Geocoding for Book Selling locations of different Authors (John Marston in this notebook)

## Outline: 
1. Geocoding Single Address
2. Data Cleaning
3. Counting and Plotting

In [2]:
import pandas as pd
import geopandas as gpd
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import matplotlib.pyplot as plt
import folium
from folium.plugins import FastMarkerCluster

### Geocoding Single Address 

In [3]:
#Finding Location of london for future map drawing

locator = Nominatim(user_agent="myGeocoder")
location = locator.geocode("St.Paul's Cathedral, London, United Kingdom")

london = locator.geocode("London, United Kingdom")

In [4]:
print(location.address)
print("Latitude = {}, Longitude = {}".format(location.latitude, location.longitude))
print("Latitude = {}, Longitude = {}".format(london.latitude, london.longitude))

St Paul's Cathedral, Saint Paul's Church Yard, Cheap, City of London, Greater London, England, EC4M 8AD, United Kingdom
Latitude = 51.51378715, Longitude = -0.09845055141832956
Latitude = 51.5073219, Longitude = -0.1276474


### Geocoding with Pandas DataFrame

In [5]:
#import text location data

df = pd.read_csv("Author Text Location - Standardized Locations _ Modern Counterparts.csv")
df.head()

,Location in 17th London,Agas,Current Location,Current Address,Notes
0,Paules Church-yarde,Center,"St. Paul's Cathedral, London, United Kingdom","St. Paul's Cathedral, St. Paul's Churchyard, L...",NaN
1,Saint Dunstan Church-yarde (East),"Also known as St. Donstone, has one in the Eas...","St Dunstan in the East Church Garden, London, ...","St Dunstan in the East Church Garden, St Dunst...",NaN
2,Saint Dunstan Church-yarde (West),"Also known as St. Donstone, has one in the Eas...","St Dunstan-in-the-West, London, United Kingdom","St Dunstan-in-the-West, 186a Fleet St, London ...",Church was moved in 19th Century in order to a...
3,Popes head Alley,"Ran south from Cornhill to Lombard Street, and...","Pope's Head Alley, London, United Kingdom","Pope's Head Alley, London EC3V 9AY, UK",NaN
4,Gratious streete,"Now known as the Gracechurch street, to the le...","Gracechurch St, London, United Kingdom","Gracechurch St, London, UK",Corresponds to portion of A10 Road that sectio...


In [6]:
#generate address, point location for each 17th century location

from geopy.extra.rate_limiter import RateLimiter

geocode = RateLimiter(locator.geocode, min_delay_seconds=1)
df['Address'] = df['Current Location'].apply(geocode)
df['point'] = df['Address'].apply(lambda loc: tuple(loc.point) if loc else None)

In [7]:
df

,Location in 17th London,Agas,Current Location,Current Address,Notes,Address,point
0,Paules Church-yarde,Center,"St. Paul's Cathedral, London, United Kingdom","St. Paul's Cathedral, St. Paul's Churchyard, L...",NaN,"(St Paul's Cathedral, Saint Paul's Church Yard...","(51.51378715, -0.09845055141832956, 0.0)"
1,Saint Dunstan Church-yarde (East),"Also known as St. Donstone, has one in the Eas...","St Dunstan in the East Church Garden, London, ...","St Dunstan in the East Church Garden, St Dunst...",NaN,"(St Dunstan in the East Church Garden, Cornhil...","(51.50963315, -0.08250508498639438, 0.0)"
2,Saint Dunstan Church-yarde (West),"Also known as St. Donstone, has one in the Eas...","St Dunstan-in-the-West, London, United Kingdom","St Dunstan-in-the-West, 186a Fleet St, London ...",Church was moved in 19th Century in order to a...,"(St Dunstan-in-the-West, 186a, Fleet Street, B...","(51.51421405, -0.11015661413978756, 0.0)"
3,Popes head Alley,"Ran south from Cornhill to Lombard Street, and...","Pope's Head Alley, London, United Kingdom","Pope's Head Alley, London EC3V 9AY, UK",NaN,None,None
4,Gratious streete,"Now known as the Gracechurch street, to the le...","Gracechurch St, London, United Kingdom","Gracechurch St, London, UK",Corresponds to portion of A10 Road that sectio...,"(Gracechurch Street, Cornhill, City of London,...","(51.5114899, -0.0854043, 0.0)"
5,Popes-head-Pallace,Assuming that it is Pope's Head Tavern (Palace...,"14 Cornhill, London, United Kingdom","14 Cornhill, London EC3V 3NR, UK",Closest corresponding address,"(The Banking Hall, 14, Cornhill, City of Londo...","(51.5130269, -0.08773859989220884, 0.0)"
6,Moore-field Gate,"Located North of London, the opening/gateway t...","Moorgate, London, United Kingdom","Moorgate, London EC2M 6TX, United Kingdom",NaN,"(Moorgate, Barbican Highwalks, Barbican, City ...","(51.5182516, -0.0890625, 0.0)"
7,S. Butolphes Church,"Has 4 locations (without Bishopsgate, Aldgate,...",NaN,NaN,NaN,"(Nanno, Ville d'Anaunia, Comunità della Val di...","(46.3144754, 11.0480288, 0.0)"
8,Christ Churchgate,"Gate next to Christ Church, which is nearby Pa...","Old Bailey, London, United Kingdom","Old Bailey, London EC1A 7AA, United Kingdom",Approximate Location Based on intersection of ...,"(Old Bailey, Blackfriars, City of London, Grea...","(51.5145541, -0.1024121, 0.0)"
9,Cornhill (Royal Exchange),Opened to make business more convenient for me...,"Royal Exchange, London, United Kingdom","Royal Exchange, London EC3V 3LR, United Kingdom",Now a Shopping mall w/ Bank of England across ...,"(Royal Exchange, Threadneedle Street, Cornhill...","(51.513645, -0.0872136547492246, 0.0)"


In [8]:
# split point column into latitude, longitude and altitude columns
df[['latitude', 'longitude', 'altitude']] = pd.DataFrame(df['point'].tolist(), index=df.index)
df.head()

,Location in 17th London,Agas,Current Location,Current Address,Notes,Address,point,latitude,longitude,altitude
0,Paules Church-yarde,Center,"St. Paul's Cathedral, London, United Kingdom","St. Paul's Cathedral, St. Paul's Churchyard, L...",NaN,"(St Paul's Cathedral, Saint Paul's Church Yard...","(51.51378715, -0.09845055141832956, 0.0)",51.513787,-0.098451,0.0
1,Saint Dunstan Church-yarde (East),"Also known as St. Donstone, has one in the Eas...","St Dunstan in the East Church Garden, London, ...","St Dunstan in the East Church Garden, St Dunst...",NaN,"(St Dunstan in the East Church Garden, Cornhil...","(51.50963315, -0.08250508498639438, 0.0)",51.509633,-0.082505,0.0
2,Saint Dunstan Church-yarde (West),"Also known as St. Donstone, has one in the Eas...","St Dunstan-in-the-West, London, United Kingdom","St Dunstan-in-the-West, 186a Fleet St, London ...",Church was moved in 19th Century in order to a...,"(St Dunstan-in-the-West, 186a, Fleet Street, B...","(51.51421405, -0.11015661413978756, 0.0)",51.514214,-0.110157,0.0
3,Popes head Alley,"Ran south from Cornhill to Lombard Street, and...","Pope's Head Alley, London, United Kingdom","Pope's Head Alley, London EC3V 9AY, UK",NaN,None,None,NaN,NaN,NaN
4,Gratious streete,"Now known as the Gracechurch street, to the le...","Gracechurch St, London, United Kingdom","Gracechurch St, London, UK",Corresponds to portion of A10 Road that sectio...,"(Gracechurch Street, Cornhill, City of London,...","(51.5114899, -0.0854043, 0.0)",51.511490,-0.085404,0.0


In [9]:
df.latitude.isnull().sum()

4

In [10]:
# Clear all the none latitude

df = df[pd.notnull(df["latitude"])]

### Visualization

In [20]:
#folium map display

map1 = folium.Map(
    location=[51.5073219, -0.1276474],
    tiles='StamenTerrain',
    zoom_start=12,
)

In [21]:
#draw a circle for each address in the dataset

df.apply(lambda row:folium.CircleMarker(location=[row["latitude"], row["longitude"]]).add_to(map1), axis=1)
map1

In [22]:
#save the map

map1.save("map1.html")

In [23]:
#import author data

df2 = pd.read_csv("Author Text Location - John Marston.csv")
df2.head()

,Author,Title/Work,Publisher,Location
0,John Marston,The scourge of villanie Three bookes of satyres.,Iohn Buzbie,Paules Church-yarde
1,John Marston,The metamorphosis of Pigmalions image And cert...,Edmond Matts,Fleet Street
2,John Marston,"The scourge of villanie Corrected, with the ad...",Iohn Buzbie,Paules Church-yarde
3,John Marston,Iacke Drums entertainment: or The comedie of P...,Richard Oliue,Long Lane
4,John Marston,Antonios reuenge. The second part. As it hath ...,Thomas Fisher,Saint Dunstan Church-yarde


In [40]:
#value counts of the column

data = df2['Location'].value_counts()
data = data.reset_index()
frame = pd.DataFrame(data)
frame = frame.rename(columns = {"index": "Location", "Location": "Count"})
frame.head()

,Location,Count
0,Paules Church-yarde,9
1,Ludgate,4
2,Saint Dunstan Church-yarde,2
3,Cornhill (Royal Exchange),2
4,Brittaines-Burse,2


In [ ]:
frame[['latitude', 'longitude', 'altitude']] = pd.DataFrame(frame['point'].tolist(), index=frame.index)
frame.head()

In [ ]:
map2 = folium.Map(
    location=[51.5073219, -0.1276474],
    tiles='cartodbpositron',
    zoom_start=12,
)
frame.apply(lambda row:folium.CircleMarker(location=[row["latitude"], row["longitude"]], radius = row["Count"]**2, 
                                          fill_color = "red").add_to(map2), axis=1)
map2

In [15]:
import numpy as np
data2 = [[1,1,1], [1,1,1], [1,1,1], [1,1,1]]
arr = np.array(data2)
print(np.ndim(arr))

2
